In [12]:
from collections import Counter
import os
import pandas as pd
from gensim.models import Phrases, phrases

def getTexts(folder):
    texts = {}
    for file in os.listdir(folder):
        if 'NOTES' not in file: 
                name = file.split('.')[0]
                path = os.path.join(folder,file)
                f = open(path,'r')
                data = f.readlines()
                if len(data) != 0: 
                    texts[name] = data[0]
                    f.close()
    return texts

def getgrams(txt):
    infile = open(txt,'r')
    lines = infile.readlines()
    infile.close()
    allgrams = []
    for line in lines: 
        line = line.split(':')
        ngrams = line[1].strip()
        if '_' in ngrams: 
                ngrams = ngrams.strip().split(' ')
                for n in ngrams:
                        allgrams.append(n)
        else: continue
    return allgrams 

In [13]:
bigramdata = getTexts('/Users/amycweng/Digital Humanities/charityTXT')
bigramtexts = list(bigramdata.values())
bigramnames = list(bigramdata.keys())
print(len(bigramnames))

199


In [14]:
training = []
for t in bigramtexts:
    words = t.split(' ')
    training.append(words)

bigrammodel = Phrases(training, min_count=2, threshold= 0, scoring='npmi', connector_words=phrases.ENGLISH_CONNECTOR_WORDS)
print('bigram model trained')

bigram model trained
trigram model trained


In [15]:
trigrammodel = Phrases(bigrammodel[training], min_count=2, threshold= 0, scoring='npmi', connector_words=phrases.ENGLISH_CONNECTOR_WORDS)
print('trigram model trained')

trigram model trained


In [16]:
searchwords = ['charity', 'charitie','charities','charitable','alms','almsgiving',
                'bequest','bequests','alm','provisions','poor','poverty','impouerished',
                'pouerty','pouertie','povertie','rates','rating','vagrant','vagrancy',
                'vagrants','caritas','profit','profits','talents','talent']
gramDict = {}
gramFreqs = {}
for key in searchwords: 
    gramDict[key] = []
    gramFreqs[key] = 0
    
name = 'charity'
storeGramFile = open(f'gramsEach{name}Text.txt','w')
count = 0
for idx, text in enumerate(bigramtexts):
    #for outputting to txt file, specify here
    textGrams = []
    text = text.strip().split(' ')
    bgcount = Counter(b for b in bigrammodel[text] if len(b.split("_")) > 1)
    tgcount = Counter(t for t in trigrammodel[text] if len(t.split("_")) > 2)
    for gram,freq in dict(bgcount).items():
        for s in searchwords: 
            if s in gram.split('_'):
                gramDict[s].append(gram)
                gramFreqs[s] += freq
                textGrams.append(gram)

    storeGramFile.write(f'{bigramnames[idx]}: {" ".join(textGrams)}\n')
    count += 1
    if count%20 == 0:
        print(count, "files processed")
storeGramFile.close()
with open(f'{name}Grams.txt', 'w+') as outfile: 
    for s in searchwords: 
        outfile.write(f"{s}: {gramFreqs[s]} occurrences\n\t\t{' '.join(gramDict[s])}\n")

print('processing complete')

20 files processed
40 files processed
60 files processed
80 files processed
100 files processed
120 files processed
140 files processed
160 files processed
180 files processed
processing complete
